# Transformer结构

在上一节中系统介绍了Transformer中核心结构self-attention机制，得益于其计算可以轻松的转化为矩阵运算因此凭借GPU可以进行并行加速的特性，使用self-attention机制替代RNN的方法在过去已经层出不穷，为之后在视觉领域引入基于Transformer的方法，本节将进一步介绍Transformer的整体结构框架。

##  整体结构

Transformer整体还是一种seq2seq的结构，分为encoder和devoder两个部分。根据Google于2017年提出的论文《Attention is all you need》，在每个encoder和decoder中都包含6个相同的module（decoder中的module与encoder中的有些许差异），下图展示了整体框架。

<div align="center">
    <img src=./images/Transformer整体结构.png width=60%/>
</div>

## 输入数据

输入数据的格式为一个matrix，其中每一行或每一列为输入句子中的单个单词经过embedding后的向量加上位置编码。

即：$input = word-embedding + positional-embedding$

其中单词的 Embedding 有很多种方式可以获取，例如可以采用 Word2Vec、Glove 等算法预训练得到，也可以在 Transformer 中训练得到。

进一步，虽然self-attention机制和RNN一样使用到了输入句子的全局信息（即每一个单词的预测都会考虑整个句子的其他单词），但是句子中单词间的位置关系却没有考虑到（每一次考虑到都是当前单词和整个句子的全局关系）。而这些有关单词位置的信息在NLP领域中极其重要，因此Transformer使用位置编码将单词的相对位置以及绝对位置保留下来。

参考[知乎文章：详解Transformer中第2.2节](https://zhuanlan.zhihu.com/p/338817680)，论文中的位置编码表示为PE，其维度和输入句子embedding的维度相同（为了后期进行concat），PE 可以通过训练得到，也可以使用某种公式计算得到。在 Transformer 中采用了后者，计算公式如下：

$$PE_{（pos, 2i）} = sin(pos / 10000^{2i/d})$$
$$PE_{（pos, 2i+1）} = cos(pos / 10000^{2i/d})$$

其中，pos 表示单词在句子中的位置，d 表示 PE的维度 (与词 Embedding 一样)，2i 表示偶数的维度，2i+1 表示奇数维度 (即 2i≤d, 2i+1≤d)。使用这种公式计算 PE 有以下的好处：

- 使 PE 能够适应比训练集里面所有句子更长的句子，假设训练集里面最长的句子是有 20 个单词，突然来了一个长度为 21 的句子，则使用公式计算的方法可以计算出第 21 位的 Embedding。
- 可以让模型容易地计算出相对位置，对于固定长度的间距 k，PE(pos+k) 可以用 PE(pos) 计算得到。因为 Sin(A+B) = Sin(A)Cos(B) + Cos(A)Sin(B), Cos(A+B) = Cos(A)Cos(B) - Sin(A)Sin(B)。

最终模型的输入可以表示如下图所示。

<div align = "center">
    <img src = "./images/input.png" width = 80%>
</div>

## Encoder Module

每一个Encoder Module大体由两部分组成，分别是前面的self-attention结构和后面的Feed Forward层（其中self-attention应指multihead self-attention即多头注意力机制，多头注意力机制可以使得模型区分不同位置的重点）。而Feed Forward层则是由两个简单的全连接层组成，其中第一个全连接层使用RelU作为激活函数，第二层全连接层则不使用任何激活函数，这样使得第二层全连接层相当于一个简单的线性变换，可以加快运算速度。其计算公式如下：$FFD(x) = Max(0, XW_{1}+b_1)W_{2} + b_{2}$，其中X为输入数据。

self-attention层和feed forward层之间使用add & normaliz层进行连接，其中Normalize层使用的是Layer Norm而不是Batch Norm，另外在Encoder Module中还使用了残差思想，一个Module中包含两个残差连接。

最后一个 Encoder block 输出的矩阵就是编码信息矩阵 C，这一矩阵后续会用到 Decoder 中。下图展示了Encoder的具体架构，在原论文中这样的Encoder一共有6个。

<div align = "center">
    <img src = ./images/encoder.png width = 80%>
</div>

## Decoder Module

Transformer的Decoder Module相较于Encoder相对复杂。
- 包含两个 Multi-Head Attention 层。
- 第一个 Multi-Head Attention 层采用了 Masked 操作。
- 第二个 Multi-Head Attention 层的K, V矩阵使用 Encoder 的编码信息矩阵C进行计算，而Q使用上一个 Decoder block 的输出计算
  

Decoder的输入包括三部分：encoder的编码信息矩阵C、当前输出的Embedding和相对应的Positional Embedding

参考 [知乎：Transformer详解](https://zhuanlan.zhihu.com/p/338817680]) 下面依次讲解Decoder的工作流程：

Decoder block 的第一个 Multi-Head Attention 采用了 Masked 操作，因为在翻译的过程中是顺序翻译的，即翻译完第 i 个单词，才可以翻译第 i+1 个单词。通过 Masked 操作可以防止第 i 个单词知道 i+1 个单词之后的信息。Decoder 可以在训练的过程中使用 Teacher Forcing 并且并行化训练，即将正确的单词序列 ("Begin" I have a cat) 和对应输出 (I have a cat "end") 传递到 Decoder。那么在预测第 i 个输出时，就要将第 i+1 之后的单词掩盖住，注意 Mask 操作是在 Self-Attention 的 Softmax 之前使用的。

Decoder block 第二个 Multi-Head Attention 变化不大， 主要的区别在于其中 Self-Attention 的 K, V矩阵不是使用上一个 Decoder block 的输出计算的，而是使用 Encoder 的编码信息矩阵 C 计算的。根据 Encoder 的输出 C计算得到 K, V，根据上一个 Decoder block 的输出 Z 计算 Q (如果是第一个 Decoder block 则使用输入矩阵 X 进行计算)，后续的计算方法与之前描述的一致。
这样做的好处是在 Decoder 的时候，每一位单词都可以利用到 Encoder 所有单词的信息 (这些信息无需 Mask)。

经过了两个多头注意力机制后，decoder的最后还需要经过一个feed forward层。

Transformer的最后是利用一个Softmax计算当前单词翻译成另一个单词的概率。

<div align = "center">
    <img src = ./images/decoder.png width = 60%>
</div>

## 总结

- Transformer 与 RNN 不同，可以比较好地并行训练。
- Transformer 本身是不能利用单词的顺序信息的，因此需要在输入中添加位置 Embedding，否则 Transformer 就是一个词袋模型了。
- Transformer 的重点是 Self-Attention 结构，其中用到的 Q, K, V矩阵通过输出进行线性变换得到。
- Transformer 中 Multi-Head Attention 中有多个 Self-Attention，可以捕获单词之间多种维度上的相关系数 attention score。